# Evolution of a margin

This notebook reproduces the [margin example](https://fastscape-lem.github.io/fastscapelib-fortran/#_margin_f90) provided in the fastscapelib-fortran library. It illustrates continental transport/deposition and sediment transport/deposition in submarine environments.

In [ ]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import fastscape

%matplotlib inline

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

## Import, customize and inspect the model

In [ ]:
from fastscape.models import marine_model

We use here a slightly simpler version of ``marine_model``:

In [ ]:
from fastscape.processes import LinearDiffusion, StreamPowerChannelTD


model = marine_model.update_processes({
    'spl': StreamPowerChannelTD,
    'diffusion': LinearDiffusion
}).drop_processes(
    'strati'
)

In [ ]:
model

In [ ]:
model.visualize(show_inputs=True)

## Model setup

In [ ]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'time': np.arange(0, 1e6 + 1e3, 1e3),
        'out': np.arange(0, 1e6 + 5e4, 5e4),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [101, 151],
        'grid__length': [1e5, 1.5e5],
        'boundary__status': ['fixed_value', 'core', 'looped', 'looped'],
        'init_topography': {
            'x_left': 0.75e5,
            'x_right': 0.75e5,
            'elevation_left': -1000.,
            'elevation_right': 0.
        },
        'uplift': {
            'x_position': 0.75e5,
            'rate_left': 0.,
            'rate_right': 1e-3
        },
        'flow__slope_exp': 1.,
        'spl': {
            'k_coef': 1e-5,
            'g_coef': 1.,
            'area_exp': 0.8,
            'slope_exp': 2.
        },
        'diffusion__diffusivity': 1e-2,
        'marine': {
            'ss_ratio_land': 0.5,
            'porosity_sand': 0.,
            'porosity_silt': 0.,
            'e_depth_sand': 1e3,
            'e_depth_silt': 1e3,
            'diffusivity_sand': 3e2 / 2.,
            'diffusivity_silt': 3e2,
            'layer_depth': 1e2,
        }
    },
    output_vars={
        'topography__elevation': 'out',
        'marine__ss_ratio_sea': 'out',
    }
)

in_ds

## Run the model

In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=model)

In [ ]:
out_ds

## Plot the outputs (topography and stratigraphic horizons)

In [ ]:
import hvplot.xarray
import holoviews as hv
from xshade import hillshade


ss_ratio_plot = out_ds.marine__ss_ratio_sea.hvplot.image(
    x='x', y='y', clim=(0, 1),
    width=550, height=350,
    cmap=plt.cm.viridis_r, groupby='out'
)

hillshade_plot = hillshade(out_ds, 'out').hvplot.image(
    x='x', y='y', cmap=plt.cm.gray, alpha=0.7,
    colorbar=False, hover=False, groupby='out'
)

ysections =  out_ds.topography__elevation.sel(y=[2.5e4, 5e4, 7.5e4])

sections_plot = ysections.hvplot.line(
    by='y', groupby='out', ylim=(-1e3, 1e3),
    width=550, height=200, legend='top_left',
)

hv.Layout((ss_ratio_plot * hillshade_plot) + sections_plot).cols(1)